In [45]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time
import numpy as np
import cv2


In [46]:
reachy = ReachySDK('localhost')

In [49]:
reachy.turn_off_smoothly('reachy')

In [22]:
reachy.turn_on('head')

In [38]:
reachy.head.look_at(x=0.9, y=0.05, z=-0.07, duration=1.5)

In [39]:
#down
reachy.head.look_at(x=0.4, y=0, z=-0.6, duration=1.5)

In [15]:
#Change the number of the dataset 
import os
path = '/home/reachy/repos/TicTacToe/tictactoe/images/empty'
files = os.listdir(path)


for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index+1011), '.jpg'])))

In [50]:
for f in reachy.fans.values():
    f.off()

Code for obtaining dataset

In [ ]:

from classes import PlaygroundTicTacToe
playground = PlaygroundTicTacToe()
low, high = playground.get_HSV() 

for k in range(150):
#Save images for dataset
    for i in range(9):
        boxes = playground.get_images(low, high)
        cv2.imwrite('images/new/'+str(j)+'.jpg', boxes[i])
        j=j+1
    k=k+1
    time.sleep(6)